In [11]:
import itertools
import math
import multiprocessing
import random
import time
import ast
from copy import deepcopy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import utilities
from Bot import Bot

bot = Bot(online=False, optimize=True)

INFO: Successfully Finished Getting Historical Data


In [2]:
space = 1
# [1, 12] Integers
space *= len(np.arange(1, 12, 1))+1
# [1.0030, 1.0500] 4 Decimal places
space *= len(np.arange(1.0030, 1.0500, 0.0001))+1
# [0.000, 0.050] 3 Decimal places
space *= len(np.arange(0, 0.05, 0.001))+1
# [0.9500, 0.9999] # 4 Decimal places
space *= len(np.arange(0.9500, 0.9999, 0.0001))+1
# [0.001, 1.000] 3 Decimal Places
space *= len(np.arange(0.001, 1, 0.001))+1
# [0.000, 0.999] 3 Decimal Places
space *= len(np.arange(0.001, 0.999, 0.001))+1

# These 3 need to sum to 1.0 or close to. 0.999 works too.
# [0.000, 0.999] 3 Decimal Places
space *= len(np.arange(0.001, 0.999, 0.001))+1
# [0.000, 0.999] 3 Decimal Places
space *= len(np.arange(0.001, 0.999, 0.001))+1
# [0.000, 0.999] 3 Decimal Places
space *= len(np.arange(0.001, 0.999, 0.001))+1
space

0

## Feature importance extraction via extra trees ensemble model with gridsearch

In [69]:
df = pd.DataFrame(
    [[
        10, 0.8247, 0.094, 0.9698, 0.687, 0.365, 0.362, 0.361, 0.356,
        1.9519551477388317
    ]],
    columns=list('123456789F'))

bg = [10, 0.8247, 0.094, 0.9698, 0.687, 0.365, 0.362, 0.361, 0.356]
bgpm = [(1, 12, 1), (1.0030, 1.0500, 0.001), (0, 0.05, 0.01),
        (0.9500, 0.9999, 0.001), (0.001, 1, 0.01), (0.001, 0.999, 0.01),
        (0.001, 0.999, 0.01), (0.001, 0.999, 0.01), (0.001, 0.999, 0.01)]

for i, g in enumerate(bg):
    genes = deepcopy(bg)
    print(i, len(np.arange(bgpm[i][0], bgpm[i][1] + bgpm[i][2], bgpm[i][2])))
    for mu in list(np.arange(bgpm[i][0], bgpm[i][1] + bgpm[i][2], bgpm[i][2])):
        print(mu, end="\r")
        genes[i] = mu
        utilities.set_optimized(genes[0], genes[1], genes[2], genes[3],
                                genes[4], genes[5], genes[6], genes[7],
                                genes[8])
        fitness = bot.run_backtest(utilities.COINPAIRS[0])
        df = df.append(
            pd.DataFrame([genes + [fitness]], columns=list('123456789F')))

0 12
1 49
2 6509999999999946
3 51
4 101
5 101999999999999945
6 101999999999999945
7 101999999999999945
8 101999999999999945
0.720999999999999945

limit_output extension: Maximum message size of 10000 exceeded with 10013 characters

In [70]:
df.nunique()

1     12
2     50
3      7
4     52
5    102
6    102
7    102
8    102
9    102
F     88
dtype: int64

In [71]:
# Feature Importance with Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]

model = ExtraTreesClassifier(n_estimators=1000)
model.fit(X, Y)
print(model.feature_importances_)

[0.109 0.233 0.099 0.08  0.074 0.141 0.119 0.143]


## Feature importance extraction via extra trees ensemble model using GA

In [2]:
import optimize

INFO: Successfully Finished Getting Historical Data


In [4]:
best, saved_genes = optimize.ga_optimize(seed=False, population_size=50, generations=50)

Gen    1 | Run Time 617.274 | Min 0.000999011 | P50 0.246104931 | Max 1.544372290 | Num unique genes:     250
Old Best {'genes': [4, 0, 1.5, 0.0, 1.0, 0.999, 0.884, 0.638, 0.999], 'fitness': -1} -1
New Best {'genes': [4, 0, 1.5, 0.0321, 0.786, 0.999, 0.759, 0.0, 0.701], 'fitness': 1.544372289792} [('BNBBTC', 1.544372289792)]
Gen    2 | Run Time 391.414 | Min 0.000999017 | P50 1.300569169 | Max 1.577985811 | Num unique genes:     511
Old Best {'genes': [4, 0, 1.5, 0.0321, 0.786, 0.999, 0.759, 0.0, 0.701], 'fitness': 1.544372289792} [('BNBBTC', 1.544372289792)]
New Best {'genes': [1, 1.4955, 1.472, 0.043, 0.095, 0.919, 0.341, 0.557, 0.454], 'fitness': 1.5779858106239997} [('BNBBTC', 1.5779858106239997)]
Gen    3 | Run Time 352.751 | Min 0.000999017 | P50 1.476943466 | Max 1.577985811 | Num unique genes:     775
Gen    4 | Run Time 374.043 | Min 0.000975287 | P50 1.314189455 | Max 1.577985811 | Num unique genes:    1039
Gen    5 | Run Time 345.998 | Min 0.000999012 | P50 1.404419647 | Max

In [15]:
gadf = pd.DataFrame(
    [[
        10, 0.8247, 0.094, 0.9698, 0.687, 0.365, 0.362, 0.361, 0.356,
        1.9519551477388317
    ]],
    columns=list('123456789F'))

for genes, fitness in saved_genes[0].items():
    gadf = gadf.append(
        pd.DataFrame([ast.literal_eval(genes) + [fitness]], columns=list('123456789F')))
    
gadf.head()

,1,2,3,4,5,6,7,8,9,F
0,10,0.8247,0.094,0.9698,0.687,0.365,0.362,0.361,0.356,1.951955
0,4,0.0000,1.500,0.0000,1.000,0.999,0.884,0.638,0.999,1.288714
0,3,1.5000,0.000,1.2418,0.952,0.999,0.999,0.999,0.896,0.094517
0,1,1.5000,1.500,1.3795,1.000,0.655,0.711,0.946,0.999,0.043577
0,12,0.0499,1.500,1.5000,1.000,0.000,0.567,0.963,0.000,0.000999


In [21]:
print(gadf.nunique())
gadf.describe()

1      12
2    3630
3    1272
4    4165
5     890
6     773
7     972
8     965
9     916
F    2392
dtype: int64


,1,2,3,4,5,6,7,8,9,F
count,12965.000000,12965.00000,12965.000000,12965.000000,12965.000000,12965.000000,12965.000000,12965.00000,12965.000000,12965.000000
mean,1.711300,1.33494,0.359127,0.190388,0.185423,0.913390,0.465541,0.33396,0.772220,1.364767
std,1.891381,0.33843,0.547635,0.286261,0.215708,0.180228,0.222665,0.24057,0.233295,0.339612
min,1.000000,0.00000,0.000000,0.000000,0.001000,0.000000,0.000000,0.00000,0.000000,0.000891
25%,1.000000,1.33830,0.000000,0.008400,0.082000,0.924000,0.353000,0.21500,0.686000,1.288714
50%,1.000000,1.49440,0.042000,0.105100,0.119000,0.990000,0.493000,0.27300,0.871000,1.378961
75%,1.000000,1.50000,0.472000,0.190900,0.202000,0.999000,0.536000,0.42500,0.904000,1.583703
max,12.000000,1.50000,1.500000,1.500000,1.000000,0.999000,0.999000,0.99900,0.999000,1.951955


In [28]:
#Feature Importance with Extra Trees Classifier
from sklearn.feature_selection import f_regression
array = gadf.values
X = array[:,0:9]
Y = array[:,9]

f_regression(X, Y)

(array([1102.68241633, 2842.61375451,  184.00228092, 6200.21679725,
         730.72625614, 1872.1824176 ,   41.11718766,  392.06676686,
         214.92372268]),
 array([3.93190011e-232, 0.00000000e+000, 1.24618469e-041, 0.00000000e+000,
        1.30973349e-156, 0.00000000e+000, 1.48343733e-010, 5.45123594e-086,
        2.81796474e-048]))